In [1]:
##IMPORT LIBRARIES

from newspaper import Article
import texthero as hero
import pandas as pd
from googletrans import Translator
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup
import requests
import time
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
##WEBSCRAPE NEWS SITES
#SETUP THE CONNECTIONS
##ADD MORE NEWS SITES TO GET MORE CATEGORIES/TOPICS


##this example is only working for sports/tech

url_sports = 'https://www.sapo.pt/noticias/desporto'

url_tech ='https://www.sapo.pt/noticias/tecnologia'

#CHECK CONNECTIONS
response_sports = requests.get(url_sports)
response_tech = requests.get(url_tech)

print(response_sports)
print(response_tech)

<Response [200]>
<Response [200]>


In [3]:
##MAKE A SOUP OBJECT FROM THE CONTENT

soup_sports = BeautifulSoup(response_sports.content,features="lxml")

soup_tech = BeautifulSoup(response_tech.content,features="lxml")

##WEBSCRAPE THE FIRST PAGES SO WE CAN GET THE NEWS LINK TO WEBSCRAPE ALL THE NEWS

#first - we get the box containing the links

news_sports = soup_sports.find_all('section', attrs={'class': '[ ink-grid vertical-space ] block block-latest-news-by-category'}) 
news_tech = soup_tech.find_all('section', attrs={'class': '[ ink-grid vertical-space ] block block-latest-news-by-category'})

#second - we get the links

links_sports = news_sports[0].find_all('a', attrs={'class': "[ quarter-bottom-space ] thumb tiny-one-by-one small-one-by-one"})
links_tech = news_tech[0].find_all('a', attrs={'class': "[ quarter-bottom-space ] thumb tiny-one-by-one small-one-by-one"})

#third - loop trough the linkws and append them to a list

list_links_sports = []

for sport_link in links_sports:
    link = sport_link.get('href')
    list_links_sports.append(link)
    
list_links_tech = []

for tech_link in links_tech:
    link1 = tech_link.get('href')
    print(link1)
    list_links_tech.append(link1)
    


https://pplware.sapo.pt/ciencia/vacina-covid-19-agendamento-disponivel-para-criancas-de-10-e-11-anos/
https://pplware.sapo.pt/ciencia/chucb-primeira-cirurgia-pouco-invasiva-do-punho-usando-nanoscope/
https://pplware.sapo.pt/apple/apple-lanca-ios-15-2-watchos-8-3-e-tvos-15-2-para-todos-os-utilizadores/
https://tek.sapo.pt/noticias/negocios/artigos/apple-esta-cada-vez-mais-perto-de-entrar-no-3-trillion-dollar-club
https://visao.sapo.pt/exameinformatica/noticias-ei/mercados/2021-12-13-elon-musk-e-eleito-personalidade-do-ano-pela-time/
https://marketeer.sapo.pt/dados-moveis-ou-chamadas-internacionais-vodafone-deixa-o-escolher-oferta-de-natal/
https://pplware.sapo.pt/internet/e-cliente-vodafone-vai-ter-comunicacoes-gratuitas-no-natal/
https://tek.sapo.pt/noticias/negocios/artigos/residuos-eletronicos-associacao-do-sector-ambiental-alerta-que-metas-nacionais-estao-comprometidas
https://visao.sapo.pt/exameinformatica/noticias-ei/jogos-ei/2021-12-13-sony-anuncia-novas-cores-para-dualsense-e-ta

In [4]:
##PREPROCESS THE LINKS 

sports_links_pre = []

for ele in list_links_sports:
    if ele[:5] != 'https':        
        sports_links_pre.append('https://www.sapo.pt/noticias/economia' + ele)
    else:
        sports_links_pre.append(ele)
        

tech_links_pre = []

for ele in list_links_tech:
    if ele[:5] != 'https':        
        tech_links_pre.append('https://www.sapo.pt/noticias/tecnologia' + ele)
    else:
        tech_links_pre.append(ele)


In [5]:
##GET THE ARTICLES INFORMATION USING NEWSPAPER LIBRARY

#loop to get them all at once 

sport_articles = []

for link in sports_links_pre:
    try:
        url = link
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        sport_articles.append(article.summary)
        time.sleep(random.random()*0.1)
       
    except:
        pass

In [6]:
#we do the same for the tech news

tech_articles = []

for link in tech_links_pre:
    try:
        url = link
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        tech_articles.append(article.summary)
        time.sleep(random.random()*0.1)
        
    except:
        pass

In [9]:
##PREPROCESS THE ARTICLES 

#preprocessing functions

def listToString(s):         
    str1 = " "          
    return (str1.join(s))


def cleaner(list):
    series = pd.Series(list)
    return hero.preprocessing.clean(series)

#first - clean the articles using texthero library

sport_clean = cleaner(sport_articles)

tech_clean = cleaner(tech_articles)

#second - transform the clean list in a single string

sport_string = listToString(sport_clean)

tech_string = listToString(tech_clean)


C:\Users\patri\anaconda3\envs\IronHack\lib\site-packages\texthero\preprocessing.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(r"^\d+\s|\s\d+\s|\s\d+$", " ")
C:\Users\patri\anaconda3\envs\IronHack\lib\site-packages\texthero\preprocessing.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pat, '')


In [10]:
######### TRANSLATE ARTICLES TO ENGLISH USING GOOGLE TRANSLATE (IGNORE IF THE ARTICLES ARE ALREADY IN ENGLISH) ###########


translator = Translator() #setup the translator

sport_eng = []

for i in range(0, len(sport_string),200):
    
    
    
    result = translator.translate(sport_string[i:i+200], src='pt') #change source to the articles language
    
    sport_eng.append((result.text))
    


In [11]:
tech_eng = []

for i in range(0, len(tech_string),200):
    
    result = translator.translate(tech_string[i:i+200], src='pt') #change source to the articles language

    tech_eng.append(result.text)

In [13]:
#TRANSFORM THE LISTS INTO DF AND LABEL EM

sports_df= pd.DataFrame(sport_eng,columns=['news'])

tech_df = pd.DataFrame(tech_eng,columns=['news'])

sports_df['label']='sports'

tech_df['label']='tech'

In [14]:
##MERGE BOTH DATAFRAMES

train = pd.concat([sports_df,tech_df]).reset_index()

train = train.drop('index', axis=1)



In [23]:
#SPLIT THE TRAIN/TEST DATA

from sklearn.model_selection import train_test_split

train, test = train_test_split(train, test_size=0.2)

In [29]:
##SAVE THE DATA

train.to_csv('train.csv')
test.to_csv('test.csv')